In [2]:
import pandas as pd

# Read data
df = pd.read_excel("MyFoodData.xlsx")

# Drops unrelevant columns
df = df.filter(items = ["Name", "Calories", "Protein (g)"])
df.head()

,Name,Calories,Protein (g)
0,Pillsbury Golden Layer Buttermilk Biscuits Art...,307.0,5.88
1,Pillsbury Cinnamon Rolls With Icing Refrigerat...,330.0,4.34
2,Kraft Foods Shake N Bake Original Recipe Coati...,377.0,6.10
3,George Weston Bakeries Thomas English Muffins,232.0,8.00
4,Waffles Buttermilk Frozen Ready-To-Heat,273.0,6.58


In [3]:
def calculateFitness(chromosome: list, maxWeight: int, negative: bool, penaltyFactor: float = 1.5):
    totalValue = 0
    totalWeight = 0

    for i in range(len(chromosome)):
        if chromosome[i] == 1:
            value = df.iloc[i]['Protein (g)']
            totalValue += value
            
            weight = df.iloc[i]['Calories']
            totalWeight += weight
    
    proteinWeight = 10
    # Penalty if the weight exceeds maxWeight
    if totalWeight > maxWeight:
        penalty = (totalWeight - maxWeight) ** penaltyFactor
        fitness = proteinWeight * totalValue - penalty
    # No penalty
    else:
        fitness = proteinWeight * totalValue

    if (negative):
        finalFitness = fitness
    else:
        return max(1, fitness)
        
    return finalFitness

In [4]:
import random

def generateIndividualWithLimit(chromosomeSize: int, maxCalories: int):
    chromosome = [0] * chromosomeSize
    indices = list(range(chromosomeSize))
    random.shuffle(indices)

    totalCalories = 0
    for i in indices:
        calorie = df.iloc[i]['Calories']
        if totalCalories + calorie <= maxCalories:
            chromosome[i] = 1
            totalCalories += calorie
        else:
            break
    return chromosome


In [5]:
import random 

def initializePopulation(populationSize: int, chromosomeSize: int, seed: int = None):
    population = []

    # Set seed
    if seed is not None:
        random.seed(seed)

    # Randomly generate populationSize amount of chromosomes with chromosomeSize size
    for i in range(populationSize):
        member = [random.randint(0, 1) for i in range(chromosomeSize)]
        population.append(member)
    
    random.seed(None)
    
    return population

def initializePopulationWithMaxWeight(populationSize: int, chromosomeSize: int, maxCalories: int, seed: int = None, rate: float = 1):
    population = []

    # Set seed
    if seed is not None:
        random.seed(seed)

    # Randomly generate populationSize amount of chromosomes with chromosomeSize size
    for i in range(int(populationSize * rate)):
        member = [random.randint(0, 1) for i in range(chromosomeSize)]
        population.append(member)
    
    while len(population) != populationSize:
        member = generateIndividualWithLimit(chromosomeSize, maxCalories)
        population.append(member)
        
    random.seed(None)
    
    return population



In [6]:
def sortPopulation(population: list[list], maxWeight: int, negative: bool, penaltyFactor: float = 1.5):
    return sorted(population, key=lambda individual: calculateFitness(individual, maxWeight, negative, penaltyFactor), reverse=True)


In [7]:
def rouletteWheelSelection(population: list[list], maxWeight: int, penaltyFactor: float = 1.5):
    total_fitness = 0
    fitnesses = []

    # Add all fitnesses
    for i in population:
        # Negative fitness is not allowed
        fitness = calculateFitness(i, maxWeight, False, penaltyFactor)
        fitnesses.append(fitness)
        total_fitness += fitness

    # An individual in the population have fitness / (sum of all fitnesses) chance to be selected in the Roulette Wheel Selection
    prob_sum = 0
    # Random float as roulette
    roulette = random.uniform(0, total_fitness)
    for i in range(len(population)):
        # If the prob_sum + fitness of the individual exceedes the roulette value, the roulette is in that probability area, so the individual is selected
        # Example:
        # If we have 5 individuals with fitness values 5, 10, 15, 6, 3 and roulette as 21, the individual with fitness value 15 should be selected.
        # In the first iteration of the loop prob_sum would be 5, in the second iteration 15 and in the third iteration it would be 30, exceeding the roulette value
        # Third individual from the population would be selected according to this algorithm
        prob_sum += fitnesses[i]
        if prob_sum > roulette:
            return population[i]

        

In [8]:
import sys

def tournamentSelection(population: list[list], tournamentSize: int, maxWeight: int, penaltyFactor: float = 1.5):
    # tournamentSize amount of individuals are selected for the tournament
    randomIndividuals = random.sample(range(0, len(population)), tournamentSize)
    bestFitness = -sys.maxsize - 1
    bestIndividual = -1
    #print(randomIndividuals)
    
    for i in randomIndividuals:
        # Individual with best fitness wins the tournament
        # This individual is selected
        fitness = calculateFitness(population[i], maxWeight, True, penaltyFactor)
        #print(f"Individual {i}, fitness {fitness}")
        if fitness > bestFitness:
            bestFitness = fitness
            bestIndividual = i
    return population[bestIndividual]


In [9]:
population = initializePopulation(10, 5)

bestIndividual = tournamentSelection(population, 4, 2500)
print(bestIndividual)

[0, 1, 1, 1, 0]


In [10]:
import random

def singlePointCrossover(firstParent: list, secondParent: list):

   # Selects a random crossover point
   crossOverPoint = random.randint(1, len(firstParent) - 1)

   # Selects the crossover parts from parents (From random point to end of the chromosome)
   crossOverPart1 = firstParent[crossOverPoint: len(firstParent)]
   crossOverPart2 = secondParent[crossOverPoint: len(secondParent)]

   firstOffspring = firstParent
   secondOffspring = secondParent

   # Swaps the crossover parts between parents
   firstOffspring[crossOverPoint: len(firstParent)] = crossOverPart2
   secondOffspring[crossOverPoint: len(secondParent)] = crossOverPart1

   return [firstOffspring, secondOffspring]

In [11]:
firstParent = [0, 0 , 1, 1, 0, 1]
secondParent = [1, 1, 0, 0, 0, 1]

offsprings = singlePointCrossover(firstParent, secondParent)
print(offsprings)

[[0, 0, 1, 0, 0, 1], [1, 1, 0, 1, 0, 1]]


In [12]:
def multiplePointCrossover(firstParent: list, secondParent: list):
    # Selects two random crossover points
    firstCrossOverPoint = random.randint(1, len(firstParent) - 1)
    secondCrossOverPoint = random.randint(1, len(secondParent) - 1)

    # Change the crossover point if the points are same
    while (firstCrossOverPoint == secondCrossOverPoint):
        secondCrossOverPoint = random.randint(1, len(firstParent) - 1)
    
    # If the first crossover point is at the right of the second crossover point swap the points
    if (firstCrossOverPoint > secondCrossOverPoint):
        firstCrossOverPoint, secondCrossOverPoint = secondCrossOverPoint, firstCrossOverPoint
    
    # Select the crossover parts from point1 to point2
    crossOverPart1 = firstParent[firstCrossOverPoint: secondCrossOverPoint + 1]
    crossOverPart2 = secondParent[firstCrossOverPoint: secondCrossOverPoint + 1]

    firstOffspring = firstParent
    secondOffspring = secondParent

    # Swap the crossover parts between parents
    firstOffspring[firstCrossOverPoint: secondCrossOverPoint + 1] = crossOverPart2
    secondOffspring[firstCrossOverPoint: secondCrossOverPoint + 1] = crossOverPart1

    return [firstOffspring, secondOffspring]

    


In [13]:
def uniformCrossover(firstParent: list, secondParent: list, maxWeight: int, penaltyFactor: float = 1.5):
    firstFitness = calculateFitness(firstParent, maxWeight, True, penaltyFactor)
    secondFitness = calculateFitness(secondParent, maxWeight, True, penaltyFactor)

    minFitness = min(firstFitness, secondFitness)
    if minFitness <= 0:
        shift = abs(minFitness) + 1
        firstFitness += shift
        secondFitness += shift

    if (firstFitness == 0 and secondFitness == 0):
        probability = 0.5
    else:
        # Probability in the uniform crossover is fp_1 / (fp_1 + fp_2), where fp_1 is the fitness of the first parent and fp_2 is the fitness of the second parent
        probability = firstFitness / (firstFitness + secondFitness)
    
    firstOffspring = []
    secondOffspring = []
    i = 0
    length = len(firstParent)
    while (len(firstOffspring) != length):
        randomProb = random.random()
        # First chromosome have the chance fp_1 / (fp_1 + fp_2) to the get the gene from first parent.
        # If the probability is higher than the random value, first offspring gets the gene from first parent
        # This repeats for all of the genes (length of the chromosome)
        if (randomProb < probability):
            firstOffspring.append(firstParent[i])
            secondOffspring.append(secondParent[i])
        else:
            firstOffspring.append(secondParent[i])
            secondOffspring.append(firstParent[i])
        i += 1

    return [firstOffspring, secondOffspring]



In [14]:
import random

def performCrossover(firstParent: list, secondParent: list, crossoverType: str, crossoverRate: float, maxWeight: int, penaltyFactor: float = 1.5):
    randomProb = random.random()
    if not (randomProb < crossoverRate):
        return [firstParent, secondParent]
    if crossoverRate == "single":
        return singlePointCrossover(firstParent, secondParent)
    elif crossoverRate == "multiple":
        return multiplePointCrossover(firstParent, secondParent)
    else:
        return uniformCrossover(firstParent, secondParent, maxWeight, penaltyFactor)

In [15]:
firstParent = [0, 0 , 1, 1, 0, 1]
secondParent = [1, 1, 0, 0, 0, 1]

offsprings = uniformCrossover(firstParent, secondParent, 1000)
print(offsprings)

[[0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 0, 1]]


In [16]:
def mutation(chromosome: list, rate: int):
    mutationsRate = 1 / rate
    for i in range(len(chromosome)):
        # random.random() generates a float between 0 and 1
        # If this number is less than or equal to mutation rate there is a bit flip in that gene (mutation occurs)
        if random.random() <= mutationsRate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

In [17]:
chromosome = [0, 0, 1, 1, 1, 0, 1]
print(mutation(chromosome, 10))

[0, 0, 1, 1, 1, 0, 1]


In [18]:
import numpy as np
import matplotlib.pyplot as plt

populationSize = 100
chromosomeSize = 100
seed = 42
maxWeight = 2000
#population = initializePopulation(populationSize, chromosomeSize, seed)
population = initializePopulationWithMaxWeight(populationSize, chromosomeSize, maxWeight, seed, 0.75)

def geneticAlgorithm(population: list[list], 
                     maxWeight: int, 
                     generations: int, 
                     crossoverRate: float, 
                     crossoverType: str, 
                     elitSize: int, 
                     penaltyFactor: float = 1.5):
    
    #meanFitness = []
    #fitnessList = [calculateFitness(i, maxWeight, True, penaltyFactor) for i in population]
    #mean = np.mean(fitnessList)
    #meanFitness.append(mean)
    

    for i in range(generations):
        sortedPopulation = sortPopulation(population, maxWeight, True, penaltyFactor)
        new_population = sortedPopulation[:elitSize]
        while len(new_population) != len(population):
            
            firstIndividual = tournamentSelection(population, 2, maxWeight, penaltyFactor)
            secondIndividual = tournamentSelection(population, 2, maxWeight, penaltyFactor)
        
            #firstIndividual = rouletteWheelSelection(population, maxWeight, penaltyFactor)
            #secondIndividual = rouletteWheelSelection(population, maxWeight, penaltyFactor)
            
            #offsprings = singlePointCrossover(firstIndividual, secondIndividual)
            offsprings = performCrossover(firstIndividual, secondIndividual, crossoverType, crossoverRate, maxWeight, penaltyFactor)
            #offsprings = multiplePointCrossover(firstIndividual, secondIndividual)
            offsprings[0] = mutation(offsprings[0], len(offsprings[0]))
            offsprings[1] = mutation(offsprings[1], len(offsprings[1]))

            new_population.extend(offsprings)

        population = new_population
        #print(f"Generation {i}")
        #print(population)
        
        #fitnessList = [calculateFitness(i, maxWeight, True, penaltyFactor) for i in population]
        #mean = np.mean(fitnessList)
        #meanFitness.append(mean)
    
    #x = np.arange(0, generations + 1)
    #plt.plot(x, meanFitness)
    #plt.title("Mean Fitness over iterations")
    #plt.xlabel("Iteration")
    #plt.ylabel("Mean Fitness")
    
    return population

generations = 100
elitismSize = 2
penaltyFactor = 1.3
crossoverRate = 1
crossoverType = "uniform"
new = geneticAlgorithm(population, maxWeight, generations,crossoverRate, crossoverType, elitismSize, penaltyFactor)


In [19]:
sortedNew = sortPopulation(new, maxWeight, True, penaltyFactor)

for i in sortedNew:
    fitness = calculateFitness(i, maxWeight, True, penaltyFactor)
    totalWeight = 0
    totalProtein = 0
    
    for j in range(len(i)):
        if i[j]:  
            protein = df.iloc[j]['Protein (g)']
            weight = df.iloc[j]['Calories']
            totalProtein += protein
            totalWeight += weight

    print(f"Fitness {fitness}, calorie {totalWeight}, protein {totalProtein}")


print(sortedNew[0])
for i, value in enumerate(sortedNew[0]):
    if value == 1:
        print(f"Name: {df.iloc[i]['Name']}, Protein (g): {df.iloc[i]['Protein (g)']}, Calories: {df.iloc[i]['Calories']}")

Fitness 1553.2999999999997, calorie 1999.0, protein 155.32999999999998
Fitness 1553.2999999999997, calorie 1999.0, protein 155.32999999999998
Fitness 1553.2999999999997, calorie 1999.0, protein 155.32999999999998
Fitness 1415.3, calorie 1828.0, protein 141.53
Fitness 1313.2999999999997, calorie 1287.0, protein 131.32999999999998
Fitness 1267.7, calorie 1971.0, protein 126.77
Fitness 1265.3, calorie 1511.0, protein 126.53
Fitness 1264.6998492873602, calorie 2015.0, protein 129.85
Fitness 1222.3, calorie 1869.0, protein 122.23
Fitness 1215.5, calorie 1994.0, protein 121.55
Fitness 1208.3, calorie 1804.0, protein 120.83
Fitness 1192.3, calorie 1434.0, protein 119.23
Fitness 1184.5, calorie 1765.0, protein 118.45
Fitness 1179.5, calorie 1871.0, protein 117.95
Fitness 1172.2, calorie 1865.0, protein 117.22
Fitness 1159.3, calorie 1571.0, protein 115.92999999999999
Fitness 1141.3, calorie 1685.0, protein 114.13
Fitness 1090.3, calorie 1475.0, protein 109.03
Fitness 1082.3, calorie 1236.0, pr

In [20]:
import numpy as np
import matplotlib.pyplot as plt

populationSize = 100
chromosomeSize = 100
seed = 42
maxWeight = 2000
#population = initializePopulation(populationSize, chromosomeSize, seed)
population = initializePopulationWithMaxWeight(populationSize, chromosomeSize, maxWeight, seed, 0.75)

# This genetic algorithm uses another elitism method
def geneticAlgorithm2(population: list[list], 
                     maxWeight: int, 
                     generations: int, 
                     crossoverRate: float, 
                     crossoverType: str, 
                     penaltyFactor: float = 1.5):
    
    #meanFitness = []
    #fitnessList = [calculateFitness(i, maxWeight, True, penaltyFactor) for i in population]
    #mean = np.mean(fitnessList)
    #meanFitness.append(mean)
    
    sortedPopulation = sortPopulation(population, maxWeight, True, penaltyFactor)
    bestIndividual = sortedPopulation[0]
    bestFitness = calculateFitness(bestIndividual, maxWeight, True, penaltyFactor)

    for i in range(generations):
        new_population = []
        while len(new_population) != len(population):
            
            firstIndividual = tournamentSelection(population, 2, maxWeight, penaltyFactor)
            secondIndividual = tournamentSelection(population, 2, maxWeight, penaltyFactor)
        
            #firstIndividual = rouletteWheelSelection(population, maxWeight, penaltyFactor)
            #secondIndividual = rouletteWheelSelection(population, maxWeight, penaltyFactor)
            
            #offsprings = singlePointCrossover(firstIndividual, secondIndividual)
            offsprings = performCrossover(firstIndividual, secondIndividual, crossoverType, crossoverRate, maxWeight, penaltyFactor)
            #offsprings = multiplePointCrossover(firstIndividual, secondIndividual)
            offsprings[0] = mutation(offsprings[0], len(offsprings[0]))
            offsprings[1] = mutation(offsprings[1], len(offsprings[1]))

            new_population.extend(offsprings)

        population = new_population
        sortedPopulation = sortPopulation(population, maxWeight, True, penaltyFactor)
        currentBestFitness = calculateFitness(sortedPopulation[0], maxWeight, True, penaltyFactor)
        if (currentBestFitness > bestFitness):
            bestIndividual = sortedPopulation[0]
            bestFitness = currentBestFitness
            
        #print(f"Generation {i}")
        #print(population)
        
        #fitnessList = [calculateFitness(i, maxWeight, True, penaltyFactor) for i in population]
        #mean = np.mean(fitnessList)
        #meanFitness.append(mean)
        
    #x = np.arange(0, generations + 1)
    #plt.plot(x, meanFitness)
    #plt.title("Mean Fitness over iterations")
    #plt.xlabel("Iteration")
    #plt.ylabel("Mean Fitness")
    
    return population, bestIndividual, bestFitness

generations = 100
penaltyFactor = 1.3
crossoverRate = 1
crossoverType = "uniform"
new, bestInd, bestFit = geneticAlgorithm2(population, maxWeight, generations, crossoverRate, crossoverType, penaltyFactor)


In [21]:
sortedNew = sortPopulation(new, maxWeight, True, penaltyFactor)
totalWeight = 0
totalProtein = 0
for i in range(len(bestInd)):
    if bestInd[i]:  
        protein = df.iloc[i]['Protein (g)']
        weight = df.iloc[i]['Calories']
        totalProtein += protein
        totalWeight += weight
print("Elitism")
print(f"Fitness {bestFit}, calorie {totalWeight}, protein {totalProtein}")

print("Actual")
for i in sortedNew:
    fitness = calculateFitness(i, maxWeight, True, penaltyFactor)
    totalWeight = 0
    totalProtein = 0
    
    for j in range(len(i)):
        if i[j]:  
            protein = df.iloc[j]['Protein (g)']
            weight = df.iloc[j]['Calories']
            totalProtein += protein
            totalWeight += weight

    print(f"Fitness {fitness}, calorie {totalWeight}, protein {totalProtein}")

Elitism
Fitness 1446.3, calorie 1991.0, protein 144.63
Actual
Fitness 1310.3999999999999, calorie 1545.0, protein 131.04
Fitness 1184.9999999999998, calorie 1976.0, protein 118.49999999999999
Fitness 1182.8, calorie 1969.0, protein 118.27999999999999
Fitness 1153.8, calorie 1993.0, protein 115.38
Fitness 1135.6999999999998, calorie 1581.0, protein 113.57
Fitness 1132.8999999999999, calorie 1993.0, protein 113.28999999999999
Fitness 1111.1, calorie 1756.0, protein 111.11
Fitness 1106.4, calorie 1982.0, protein 110.64
Fitness 1101.3, calorie 1887.0, protein 110.13
Fitness 1072.3999999999999, calorie 1911.0, protein 107.24
Fitness 1069.2, calorie 1954.0, protein 106.92
Fitness 1066.5085594719806, calorie 2042.0, protein 119.53999999999999
Fitness 1066.1999999999998, calorie 1908.0, protein 106.61999999999999
Fitness 1057.6, calorie 1640.0, protein 105.75999999999999
Fitness 1056.2, calorie 1834.0, protein 105.62
Fitness 1049.3, calorie 1417.0, protein 104.92999999999999
Fitness 1044.89999

In [22]:
numberOfExperiments = 5

populationSize = 100
chromosomeSize = 100
seed = None
maxWeight = 2000
population = initializePopulationWithMaxWeight(populationSize, chromosomeSize, maxWeight, seed, 0.75)
generations = 100
elitismSize = 2
penaltyFactor = 1.2
crossoverRate = 0.95
crossoverType = "uniform"

with open("exp1.txt", "w", encoding="utf-8") as file:
    pass

fitnessList = []
proteinList = []
calorieList = []
feasibleAmount = []
infeasibleAmount = []

for i in range(numberOfExperiments):
    with open("exp1.txt", "a", encoding="utf-8") as file:
        file.write(f"Experiment {i}:" + "\n")
        file.write("-" * 10)
        file.write("\n")

    currentPopulation = geneticAlgorithm(population, maxWeight, generations, crossoverRate, crossoverType, elitismSize, penaltyFactor)
    sortedNew = sortPopulation(currentPopulation, maxWeight, True, penaltyFactor)
    
    numFeasible = 0
    numInfeasible = 0
    
    for j in sortedNew:
        totalWeight = 0
        for k in range(len(j)):
            if j[k]:  
                weight = df.iloc[k]['Calories']
                totalWeight += weight
        if (totalWeight > maxWeight):
            numInfeasible += 1
        else:
            numFeasible += 1

    feasibleAmount.append(numFeasible)
    infeasibleAmount.append(numInfeasible)
    totalWeight = 0
    totalProtein = 0

    for k in range(5):
        totalWeight = 0
        totalProtein = 0
        for l in range(len(sortedNew[k])):
            if sortedNew[k][l]:  
                protein = df.iloc[l]['Protein (g)']
                weight = df.iloc[l]['Calories']
                totalProtein += protein
                totalWeight += weight
        
        fitness = calculateFitness(sortedNew[k], maxWeight, True, penaltyFactor)
        
        if k == 0:
            fitnessList.append(fitness)
            proteinList.append(totalProtein)
            calorieList.append(totalWeight)

        with open("exp1.txt", "a", encoding="utf-8") as file:
            file.write(f"Individual {k}" + "\n")
            file.write("-" * 10)
            file.write("\n")
            file.write("Fitness of  individual " + str(fitness) + "\n")
            file.write("Protein of individual " + str(totalProtein) + "\n")
            file.write("Weight of individual " + str(totalWeight) + "\n")
            file.write("\n")

  
    with open("exp1.txt", "a", encoding="utf-8") as file:
        file.write("Number of feasible " + str(numFeasible) + "\n")
        file.write("Number of infeasible " + str(numInfeasible) + "\n")
        file.write("\n")

with open("exp1.txt", "a", encoding="utf-8") as file:
    file.write(f"Mean Protein of best {np.mean(proteinList)} " + "\n")
    file.write(f"Mean Fitness of best {np.mean(fitnessList)} " + "\n")
    file.write(f"Mean Calorie of best {np.mean(calorieList)} " + "\n")
    file.write(f"Mean Amount of feasible Solutions {np.mean(feasibleAmount)} " + "\n")
    file.write(f"Mean Amount of infeasible Solutions {np.mean(infeasibleAmount)} " + "\n")








In [ ]:
numberOfExperiments = 5

populationSize = 100
chromosomeSize = 100
seed = None
maxWeight = 2000
population = initializePopulationWithMaxWeight(populationSize, chromosomeSize, maxWeight, seed, 0.75)
generations = 100
elitismSize = 2
penaltyFactor = 1.2
crossoverRate = 0.95
crossoverType = "uniform"

with open("exp2.txt", "w", encoding="utf-8") as file:
    pass

fitnessList = []
proteinList = []
calorieList = []
feasibleAmount = []
infeasibleAmount = []

for i in range(numberOfExperiments):
    with open("exp2.txt", "a", encoding="utf-8") as file:
        file.write(f"Experiment {i}:" + "\n")
        file.write("-" * 10)
        file.write("\n")

    currentPopulation, bestInd, bestFit = geneticAlgorithm2(population, maxWeight, generations, crossoverRate, crossoverType, penaltyFactor)
    sortedNew = sortPopulation(currentPopulation, maxWeight, True, penaltyFactor)
    numFeasible = 0
    numInfeasible = 0
    
    for j in sortedNew:
        totalWeight = 0
        for k in range(len(j)):
            if j[k]:  
                weight = df.iloc[k]['Calories']
                totalWeight += weight
        if (totalWeight > maxWeight):
            numInfeasible += 1
        else:
            numFeasible += 1

    feasibleAmount.append(numFeasible)
    infeasibleAmount.append(numInfeasible)
    totalWeight = 0
    totalProtein = 0
    
    for k in range(len(bestInd)):
        if bestInd[k]:  
            protein = df.iloc[k]['Protein (g)']
            weight = df.iloc[k]['Calories']
            totalProtein += protein
            totalWeight += weight

    fitnessList.append(bestFit)
    proteinList.append(totalProtein)
    calorieList.append(totalWeight)

    with open("exp2.txt", "a", encoding="utf-8") as file:
        file.write("Best Individual" + "\n")
        file.write("-" * 10)
        file.write("\n")
        file.write("Fitness of  individual " + str(bestFit) + "\n")
        file.write("Protein of individual " + str(totalProtein) + "\n")
        file.write("Weight of individual " + str(totalWeight) + "\n")
        file.write("\n")

    for k in range(5):
        totalWeight = 0
        totalProtein = 0
        for l in range(len(sortedNew[k])):
            if sortedNew[k][l]:  
                protein = df.iloc[l]['Protein (g)']
                weight = df.iloc[l]['Calories']
                totalProtein += protein
                totalWeight += weight

        fitness = calculateFitness(sortedNew[k], maxWeight, True, penaltyFactor)
        
        with open("exp2.txt", "a", encoding="utf-8") as file:
            file.write(f"Individual {k}" + "\n")
            file.write("-" * 10)
            file.write("\n")
            file.write("Fitness of  individual " + str(fitness) + "\n")
            file.write("Protein of individual " + str(totalProtein) + "\n")
            file.write("Weight of individual " + str(totalWeight) + "\n")
            file.write("\n")

  
    with open("exp2.txt", "a", encoding="utf-8") as file:
        file.write("Number of feasible " + str(numFeasible) + "\n")
        file.write("Number of infeasible " + str(numInfeasible) + "\n")
        file.write("\n")

with open("exp2.txt", "a", encoding="utf-8") as file:
    file.write(f"Mean Protein of best {np.mean(proteinList)} " + "\n")
    file.write(f"Mean Fitness of best {np.mean(fitnessList)} " + "\n")
    file.write(f"Mean Calorie of best {np.mean(calorieList)} " + "\n")
    file.write(f"Mean Amount of feasible Solutions {np.mean(feasibleAmount)} " + "\n")
    file.write(f"Mean Amount of infeasible Solutions {np.mean(infeasibleAmount)} " + "\n")




